In [9]:
"""
testing.ipynb

File for performing testing to implement lottery ticket experiments.

Authors: Jordan Bourdeau, Casey Forey
Date Created: 3/8/24
"""

%load_ext tensorboard
import functools
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras

from src.harness import constants as C
from src.harness.dataset import download_data, load_and_process_mnist
from src.harness.experiment import experiment
from src.harness.model import create_model, LeNet300, load_model
from src.harness.pruning import prune_by_percent
from src.harness.training import train, TrainingRound
from src.lottery_ticket.foundations import paths

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [21]:
X_train, Y_train, X_test, Y_test = load_and_process_mnist()

# Create a model with the same architecture using all Keras components to check its accuracy with the same parameters
def create_lenet_300_100(input_shape: tuple[int, ...], num_classes: int):
    """
    Simple hardcoded class definition for creating the sequential Keras equivalent to LeNet-300-100.
    """
    model = keras.Sequential(name="LeNet-300-100")
    model.add(keras.layers.Flatten(input_shape=input_shape))
    model.add(keras.layers.Dense(300, activation='relu'))
    model.add(keras.layers.Dense(100, activation='relu'))
    model.add(keras.layers.Dense(num_classes, activation='softmax'))
    return model

num_classes: int = 10
input_shape: tuple[int, ...] = X_train[0].shape
keras_model: keras.Model = create_lenet_300_100(input_shape, num_classes)
keras_model.compile(loss=keras.losses.CategoricalCrossentropy(), optimizer=C.OPTIMIZER(), metrics=['accuracy'])
# Train the model
history = keras_model.fit(X_train, Y_train,
                             batch_size=128,
                             epochs=60,
                             verbose=1,
                             validation_data=(X_test, Y_test))

# Evaluate the model on the test set
test_loss, test_accuracy = keras_model.evaluate(X_test, Y_test, verbose=0)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')

Epoch 1/60
469/469 [==============================] - 1s 2ms/step - loss: 0.4215 - accuracy: 0.8827 - val_loss: 0.2239 - val_accuracy: 0.9339
Epoch 2/60
469/469 [==============================] - 1s 1ms/step - loss: 0.1969 - accuracy: 0.9433 - val_loss: 0.1752 - val_accuracy: 0.9487
Epoch 3/60
469/469 [==============================] - 1s 1ms/step - loss: 0.1433 - accuracy: 0.9585 - val_loss: 0.1372 - val_accuracy: 0.9603
Epoch 4/60
469/469 [==============================] - 1s 1ms/step - loss: 0.1135 - accuracy: 0.9670 - val_loss: 0.1170 - val_accuracy: 0.9640
Epoch 5/60
469/469 [==============================] - 1s 2ms/step - loss: 0.0930 - accuracy: 0.9732 - val_loss: 0.1026 - val_accuracy: 0.9694
Epoch 6/60
469/469 [==============================] - 1s 2ms/step - loss: 0.0773 - accuracy: 0.9772 - val_loss: 0.0964 - val_accuracy: 0.9693
Epoch 7/60
469/469 [==============================] - 1s 1ms/step - loss: 0.0660 - accuracy: 0.9813 - val_loss: 0.0867 - val_accuracy: 0.9724
Epoch 

KeyboardInterrupt: 

In [2]:
TESTING_RANDOM_SEED: int = 999

In [3]:
# Create a model
model = create_model(TESTING_RANDOM_SEED, X_train, Y_train)
initial_weights1: dict[str: np.array] = model.get_current_weights()

In [4]:
# Test Training a model
optimizer = C.OPTIMIZER()
make_dataset: callable = load_and_process_mnist
round: TrainingRound = train(make_dataset, model, TESTING_RANDOM_SEED, optimizer, C.TEST_TRAINING_ITERATIONS)

for i in range(3):
    key: str = f'layer{i}'
    # Sanity check that the initial weights are correct
    assert np.array_equal(initial_weights1[key], round.initial_weights[key])
    # Verify the final weights are different from initial weights
    assert not np.array_equal(round.initial_weights[key], round.final_weights[key])


Iteration 1/10, Loss: 2.3420140743255615
Iteration 2/10, Loss: 2.257554769515991
Iteration 3/10, Loss: 2.1908860206604004
Iteration 4/10, Loss: 2.1296677589416504
Iteration 5/10, Loss: 2.0702004432678223
Iteration 6/10, Loss: 2.0107617378234863
Iteration 7/10, Loss: 1.950530767440796
Iteration 8/10, Loss: 1.8891801834106445
Iteration 9/10, Loss: 1.8266687393188477
Iteration 10/10, Loss: 1.763145089149475


In [5]:
# Try creating a model from the initial weights as a preset
make_model: callable = functools.partial(LeNet300, TESTING_RANDOM_SEED)
percents: dict[str: float] = {key: 0.5 for key in round.final_weights}
starting_masks: dict[str: np.ndarray] = {f'layer{i}': np.ones(round.initial_weights[f'layer{i}'].shape) for i in range(3)}

# Create pruned masks
masks = prune_by_percent(C.TEST_PRUNING_PERCENTS, starting_masks, round.final_weights)

# This is transforming initial weights into a tensor when it should be Numpy?
# Passing in masks is the issue
model = make_model(X_train, Y_train, presets=round.initial_weights, masks=masks)

# Sanity check that the weights are correctly loaded and are masked off accordingly
for i in range(3):
    key = f'layer{i}'
    layer_weights = model.get_current_weights()[key]
    layer_mask = model.masks[key]
    expected_weights: np.ndarray = round.initial_weights[key] * layer_mask
    assert np.array_equal(expected_weights, layer_weights), f'Expected {expected_weights} but received {layer_weights}'
    assert np.array_equal(model.masks[key], masks[key])

# Save the tensors storing the actual weight values (these include the masked off weights)
pretrained_weights = model.weights.copy()

# Try doing a a simulated round of pruning
round2: TrainingRound = train(make_dataset, model, 1, optimizer, C.TEST_TRAINING_ITERATIONS)

# Make sure the masked off weights don't receive any updates in the actual tensorflow tensor
trained_weights = model.weights

# Compare the masked weights before and after training
for i in range(3):
    key = f'layer{i}'
    pretrained_layer_weights = pretrained_weights[key]
    trained_layer_weights = trained_weights[key]
    
    # Invert the mask, to only look at the weights which WERE masked off
    inverted_mask: np.ndarray = 1 - masks[key]
    masked_pretrained_weights = pretrained_layer_weights * inverted_mask
    masked_trained_weights = trained_layer_weights * inverted_mask
    
    # Assert that the masked weights remain unchanged after training
    assert np.array_equal(masked_pretrained_weights, masked_trained_weights), f'Weights changed after training for layer {key}'
    assert not np.array_equal(round2.initial_weights, round.final_weights)


Iteration 1/10, Loss: 2.280717134475708
Iteration 2/10, Loss: 2.204838991165161
Iteration 3/10, Loss: 2.1418585777282715
Iteration 4/10, Loss: 2.0832467079162598
Iteration 5/10, Loss: 2.026040554046631
Iteration 6/10, Loss: 1.9689557552337646
Iteration 7/10, Loss: 1.9113845825195312
Iteration 8/10, Loss: 1.8531246185302734
Iteration 9/10, Loss: 1.794109582901001
Iteration 10/10, Loss: 1.7345093488693237


In [6]:
MODEL_INDEX: int = 0
PRUNING_STEP: int = 5

# Get initial weights
dir: str = f'models/model_{MODEL_INDEX}/initial/'
weight_files = [paths.weights(dir) + f'/layer{i}.npy' for i in range(3)]
mask_files = [paths.masks(dir) + f'/layer{i}.npy' for i in range(3)]

layer_weights = {f'layer{i}': np.load(layer) for i, layer in enumerate(weight_files)}
masks = {f'layer{i}': np.load(layer) for i, layer in enumerate(mask_files)}
# Test loading a model
model: LeNet300 = load_model(MODEL_INDEX, PRUNING_STEP, True)

for i in range(3):
    key: str = f'layer{i}'
    # Verify all the layer weights match
    assert np.array_equal(model.weights[key], layer_weights[key])
    # Verify all masks are 1s
    assert np.sum(masks[key]) == masks[key].size


In [7]:
# Test pruning
print([(key, layer.shape) for key, layer in layer_weights.items()])
percents: dict[str: float] = {key: 0.5 for key in layer_weights}
new_masks: dict[str, np.array] = prune_by_percent(percents, masks, layer_weights)
for key in new_masks:
    new_mask: np.array = new_masks[key]
    old_mask: np.array = masks[key]
    assert (old_mask.sum() / 2 - new_mask.sum()) <= 1, f'Doesn\'t match for key {key}'

[('layer0', (784, 300)), ('layer1', (300, 100)), ('layer2', (100, 10))]


In [8]:
# Test experiment
make_dataset: callable = load_and_process_mnist
# Make partial function application giving the model its random seed
make_model: callable = functools.partial(LeNet300, 999)
train_model: callable = functools.partial(train, iterations=C.TEST_TRAINING_ITERATIONS)
prune_masks: callable = functools.partial(prune_by_percent, C.PRUNING_PERCENTS)
experiment(make_dataset, make_model, train_model, prune_masks, C.TEST_PRUNING_STEPS)

Pruning Step 0
Iteration 1/10, Loss: 2.3612027168273926
Iteration 2/10, Loss: 2.258298635482788
Iteration 3/10, Loss: 2.1781187057495117
Iteration 4/10, Loss: 2.1064586639404297
Iteration 5/10, Loss: 2.037797689437866
Iteration 6/10, Loss: 1.9696766138076782
Iteration 7/10, Loss: 1.901034951210022
Iteration 8/10, Loss: 1.8315355777740479
Iteration 9/10, Loss: 1.7611823081970215
Iteration 10/10, Loss: 1.6903091669082642
Pruning Step 1
Iteration 1/10, Loss: 2.2866768836975098
Iteration 2/10, Loss: 2.1959917545318604
Iteration 3/10, Loss: 2.1210360527038574
Iteration 4/10, Loss: 2.052361011505127
Iteration 5/10, Loss: 1.9861609935760498
Iteration 6/10, Loss: 1.920701026916504
Iteration 7/10, Loss: 1.855150580406189
Iteration 8/10, Loss: 1.7892708778381348
Iteration 9/10, Loss: 1.7231773138046265
Iteration 10/10, Loss: 1.6571670770645142
Pruning Step 2
Iteration 1/10, Loss: 2.2150192260742188
Iteration 2/10, Loss: 2.132519245147705
Iteration 3/10, Loss: 2.0611836910247803
Iteration 4/10, L